<img style="width:450px;" src="https://durhamcollege.ca/wp-content/uploads/ai-hub-header.jpg" alt="DC Logo"/>

# LESSON 11 - Decision Trees and Random Forest

## <span style="color: green">OVERVIEW</span>

- ref: http://benalexkeen.com/decision-tree-classifier-in-python-using-scikit-learn/
- dataset: https://www.kaggle.com/c/titanic/data

### Notes:
   - Please include pictures wherever you feel there is too much text to break things up a bit

## <span style="color: green">SECTION 1</span>


### <span style="color: blue">Decision Trees</span>

Decision Trees can be used as classifier or regression models.

A tree structure is constructed that breaks the dataset down into smaller subsets eventually resulting in a prediction. There are decision nodes that partition the data and leaf nodes that give the prediction that can be followed by traversing simple IF..AND..AND….THEN logic down the nodes.

The root node (the first decision node) partitions the data based on the most influential feature partitioning. There are 2 measures for this, Gini Impurity and Entropy.

### Entropy
The root node (the first decision node) partitions the data using the feature that provides the most information gain.

Information gain tells us how important a given attribute of the feature vectors is.

It is calculated as:

$$\text{Information Gain} = \text{entropy(parent)} – \text{[average entropy(children)]}$$

Where entropy is a common measure of target class impurity, given as:

$$Entropy = \Sigma_i – p_i \log_2 p_i$$

where i is each of the target classes.

### Gini Impurity
Gini Impurity is another measure of impurity and is calculated as follows:

$$Gini = 1 – \Sigma_i p_i^2$$

Gini impurity is computationally faster as it doesn’t require calculating logarithmic functions, though in reality which of the two methods is used rarely makes too much of a difference.

### Predicting Survival in the Titanic Data Set
We’ll be using a decision tree to make predictions about the Titanic data set from Kaggle. This data set provides information on the Titanic passengers and can be used to predict whether a passenger survived or not.

In [1]:
import pandas as pd

df = pd.read_csv('train.csv', index_col='PassengerId')

In [2]:
print(df.head())

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked  
PassengerId                                                          
1                1      0         A/5 21171   7.2500   NaN        S

We will be using Pclass, Sex, Age, SibSp (Siblings aboard), Parch (Parents/children aboard), and Fare to predict whether a passenger survived.

In [3]:
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

We need to convert ‘Sex’ into an integer value of 0 or 1.

In [4]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

We will also drop any rows with missing values.

In [5]:
df = df.dropna()

X = df.drop('Survived', axis=1)
y = df['Survived']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
from sklearn import tree

model = tree.DecisionTreeClassifier()

Let’s take a look at our model’s attributes

In [8]:
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Defining some of the attributes like max_depth, max_leaf_nodes, min_impurity_split, and min_samples_leaf can help prevent overfitting the model to the training data.

First we fit our model using our training data.

In [9]:
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Then we score the predicted output from model on our test data against our ground truth test data.

In [10]:
y_predict = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.82681564245810057

We see an accuracy score of ~81.01%, which is significantly better than 50/50 guessing.

Let’s also take a look at our confusion matrix:

In [11]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

Predicted Not Survival  Predicted Survival
True Not Survival                      98                  14
True Survival                          17                  50

OPTIONAL
Download the Graphviz tool to visualize the data.
https://graphviz.gitlab.io/download/

In [12]:
#tree.export_graphviz(model.tree_, out_file='tree.dot', feature_names=X.columns)

We can then convert this dot file to a png file.

In [13]:
from subprocess import call

#call(['dot', '-T', 'png', 'tree.dot', '-o', 'tree.png'])

We can then view our tree, which looks something like this:

<img src="http://benalexkeen.com/wp-content/uploads/2017/05/tree.png" alt="Decision Tree" />

The root node, with the most information gain, tells us that the biggest factor in determining survival is Sex.

If we zoom in on some of the leaf nodes, we can follow some of the decisions down.

We have already zoomed into the part of the decision tree that describes males, with a ticket lower than first class, that are under the age of 10.

<img src="http://benalexkeen.com/wp-content/uploads/2017/05/tree_leaf_node.png" alt="Leaf Node" />

The impurity is the measure as given at the top by Gini, the samples are the number of observations remaining to classify and the value is the how many samples are in class 0 (Did not survive) and how many samples are in class 1 (Survived).

Let’s follow this part of the tree down, the nodes to the left are True and the nodes to the right are False:

1. We see that we have 19 observations left to classify: 9 did not survive and 10 did.
2. From this point the most information gain is how many siblings (SibSp) were aboard.

        A. 9 out of the 10 samples with less than 2.5 siblings survived.
        B. This leaves 10 observations left, 9 did not survive and 1 did.
    
3. 6 of these children that only had one parent (Parch) aboard did not survive.
4. None of the children aged > 3.5 survived
5. Of the 2 remaining children, the one with > 4.5 siblings did not survive.

## <span style="color: green">SECTION 2</span>


### <span style="color: blue">Random Forest</span>

In [14]:
# remove warnings
import warnings
#warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
pd.options.display.max_rows = 100

In [15]:
def status(feature):
    print('Processing', feature, ':OK')

In [16]:
def get_combined_data():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    targets = train.Survived # extracting and removing the targets from training data
    train.drop(['Survived'], 1, inplace=True)
    
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop('index', inplace=True, axis=1)
    return combined

In [17]:
combined = get_combined_data()

In [18]:
combined.shape

(1309, 11)

Now that we have the dataset extracted.

Let's extract the Passenger Titles

In [19]:
def get_titles():
    global combined
    combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    Title_Dictionary = {
        'Capt':    'Officer',
        'Col':     'Officer',
        'Major':   'Officer',
        'Jonkheer':'Royalty',
        'Don':     'Royalty',
        'Sir':     'Royalty',
        'Dr':      'Officer',
        'Rev':     'Officer',
        'the Countess':'Royalty',
        'Dona':    'Royalty',
        'Mme':'Mrs',
        'Mlle':'Miss',
        'Ms':'Mrs',
        'Mr':'Mr',
        'Mrs':'Mrs',
        'Miss':'Miss',
        'Master':'Master',
        'Lady':'Royalty'
    }
    combined['Title'] = combined.Title.map(Title_Dictionary)
    combined.drop('Name', 1, inplace=True)

In [20]:

get_titles()
combined.head(5)

PassengerId  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0            1       3    male  22.0      1      0         A/5 21171   7.2500   
1            2       1  female  38.0      1      0          PC 17599  71.2833   
2            3       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3            4       1  female  35.0      1      0            113803  53.1000   
4            5       3    male  35.0      0      0            373450   8.0500   

  Cabin Embarked Title  
0   NaN        S    Mr  
1   C85        C   Mrs  
2   NaN        S  Miss  
3  C123        S   Mrs  
4   NaN        S    Mr

Process the Passenger Ages

In [21]:
grouped = combined.groupby(['Sex','Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch      Fare
Sex    Pclass Title                                             
female 1      Miss           529.5  30.0    0.0    0.0   99.9625
              Mrs            853.5  45.0    1.0    0.0   78.1125
              Officer        797.0  49.0    0.0    0.0   25.9292
              Royalty        760.0  39.0    0.0    0.0   86.5000
       2      Miss           606.5  20.0    0.0    0.0   20.2500
              Mrs            533.0  30.0    1.0    0.0   26.0000
       3      Miss           603.5  18.0    0.0    0.0    8.0500
              Mrs            668.5  31.0    1.0    1.0   15.5000
male   1      Master         803.0   6.0    1.0    2.0  134.5000
              Mr             634.0  41.5    0.0    0.0   47.1000
              Officer        678.0  52.0    0.0    0.0   37.5500
              Royalty        600.0  40.0    0.0    0.0   27.7208
       2      Master         550.0   2.0    1.0    1.0   26.0000
              Mr             723.5  30.0    0.0    0.0   13.0000
              Officer        513.0  41.5    0.0    0.0   13.0000
       3      Master         789.0   6.0    3.0    1.0   22.3583
              Mr             640.5  26.0    0.0    0.0    7.8958

In [22]:
def process_age():
    
    global combined
    
    # a function that fills the missing values of the Age variable
    
    def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26
    combined.Age = combined.apply(lambda r: fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)
    status('age')

In [23]:
process_age()
combined.info()

Processing age :OK
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [24]:
def process_names():
    global combined
    titles_dummies = pd.get_dummies(combined['Title'], prefix='Title')
    combined = pd.concat([combined, titles_dummies], axis=1)
    combined.drop('Title', axis=1, inplace=True)
    status('Name')

In [25]:
process_names()
combined.head(5)

Processing Name :OK


PassengerId  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0            1       3    male  22.0      1      0         A/5 21171   7.2500   
1            2       1  female  38.0      1      0          PC 17599  71.2833   
2            3       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3            4       1  female  35.0      1      0            113803  53.1000   
4            5       3    male  35.0      0      0            373450   8.0500   

  Cabin Embarked  Title_Master  Title_Miss  Title_Mr  Title_Mrs  \
0   NaN        S             0           0         1          0   
1   C85        C             0           0         0          1   
2   NaN        S             0           1         0          0   
3  C123        S             0           0         0          1   
4   NaN        S             0           0         1          0   

   Title_Officer  Title_Royalty  
0              0              0  
1              0              0  
2              0              0  
3              0              0  
4              0              0

In [26]:
def process_fare():
    global combined
    combined.Fare.fillna(combined.Fare.mean(), inplace=True)
    status('fare')

In [27]:
process_fare()

Processing fare :OK


In [28]:
def process_embarked():
    global combined
    combined.Embarked.fillna('S', inplace=True)
    # dummy encoding
    embarked_dummies = pd.get_dummies(combined['Embarked'], prefix='Embarked')
    combined = pd.concat([combined, embarked_dummies], axis=1)
    combined.drop('Embarked', axis=1, inplace=True)
    status('Embarked')

In [29]:
process_embarked()

Processing Embarked :OK


In [30]:
def process_cabin():
    global combined
    combined.Cabin.fillna('U', inplace=True)
    # mapping each 
    combined['Cabin'] = combined['Cabin'].map(lambda c : c[0])
    # dummy encoding
    cabin_dummies = pd.get_dummies(combined['Cabin'], prefix='Cabin')
    combined = pd.concat([combined, cabin_dummies], axis=1)
    combined.drop('Cabin', axis=1, inplace=True)
    status('Cabin')

In [31]:
process_cabin()
combined.head(5)

Processing Cabin :OK


PassengerId  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0            1       3    male  22.0      1      0         A/5 21171   7.2500   
1            2       1  female  38.0      1      0          PC 17599  71.2833   
2            3       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3            4       1  female  35.0      1      0            113803  53.1000   
4            5       3    male  35.0      0      0            373450   8.0500   

   Title_Master  Title_Miss  Title_Mr  Title_Mrs  Title_Officer  \
0             0           0         1          0              0   
1             0           0         0          1              0   
2             0           1         0          0              0   
3             0           0         0          1              0   
4             0           0         1          0              0   

   Title_Royalty  Embarked_C  Embarked_Q  Embarked_S  Cabin_A  Cabin_B  \
0              0           0           0           1        0        0   
1              0           1           0           0        0        0   
2              0           0           0           1        0        0   
3              0           0           0           1        0        0   
4              0           0           0           1        0        0   

   Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_U  
0        0        0        0        0        0        0        1  
1        1        0        0        0        0        0        0  
2        0        0        0        0        0        0        1  
3        1        0        0        0        0        0        0  
4        0        0        0        0        0        0        1

In [32]:
def process_sex():
    global combined
    combined['Sex'] = combined['Sex'].map({'male':0, 'female':1})
    status('sex')

In [33]:
process_sex()

Processing sex :OK


In [34]:
def process_pclass():
    global combined
    pclass_dummies = pd.get_dummies(combined['Pclass'], prefix='Pclass')
    combined = pd.concat([combined, pclass_dummies], axis=1)
    combined.drop('Pclass', axis=1, inplace=True)
    status('pclass')

In [35]:
process_pclass()

Processing pclass :OK


In [36]:
def process_ticket():
    global combined
    # a function that extracts each prefix of the ticket, returns 'XXX' if no prefix
    def cleanTicket(ticket):
        ticket = ticket.replace('.','')
        ticket = ticket.replace('/','')
        ticket = map(lambda t : t.strip(), ticket)
        # print(type(ticket))
        ticket = list(filter(lambda t : not t.isdigit(), ticket))
        if len(ticket) > 0:
            return ticket[0]
        else:
            return 'XXX'
    # extracing dummy variables from tickets
    combined['Ticket'] = combined['Ticket'].map(cleanTicket)
    tickets_dummies = pd.get_dummies(combined['Ticket'], prefix='Ticket')
    combined = pd.concat([combined, tickets_dummies], axis=1)
    combined.drop('Ticket', inplace=True, axis=1)
    status('Ticket')

In [37]:
process_ticket()
combined.head(5)

Processing Ticket :OK


PassengerId  Sex   Age  SibSp  Parch     Fare  Title_Master  Title_Miss  \
0            1    0  22.0      1      0   7.2500             0           0   
1            2    1  38.0      1      0  71.2833             0           0   
2            3    1  26.0      0      0   7.9250             0           1   
3            4    1  35.0      1      0  53.1000             0           0   
4            5    0  35.0      0      0   8.0500             0           0   

   Title_Mr  Title_Mrs  Title_Officer  Title_Royalty  Embarked_C  Embarked_Q  \
0         1          0              0              0           0           0   
1         0          1              0              0           1           0   
2         0          0              0              0           0           0   
3         0          1              0              0           0           0   
4         1          0              0              0           0           0   

   Embarked_S  Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  \
0           1        0        0        0        0        0        0        0   
1           0        0        0        1        0        0        0        0   
2           1        0        0        0        0        0        0        0   
3           1        0        0        1        0        0        0        0   
4           1        0        0        0        0        0        0        0   

   Cabin_T  Cabin_U  Pclass_1  Pclass_2  Pclass_3  Ticket_A  Ticket_C  \
0        0        1         0         0         1         1         0   
1        0        0         1         0         0         0         0   
2        0        1         0         0         1         0         0   
3        0        0         1         0         0         0         0   
4        0        1         0         0         1         0         0   

   Ticket_F  Ticket_L  Ticket_P  Ticket_S  Ticket_W  Ticket_XXX  
0         0         0         0         0         0           0  
1         0         0         1         0         0           0  
2         0         0         0         1         0           0  
3         0         0         0         0         0           1  
4         0         0         0         0         0           1

In [38]:
def process_family():
    global combined
    combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1
    
    combined['Singleton'] = combined['FamilySize'].map(lambda s : 1 if s == 1 else 0)
    combined['SmallFamily'] = combined['FamilySize'].map(lambda s : 1 if 2<=s<=4 else 0)
    combined['BigFamily'] = combined['FamilySize'].map(lambda s : 1 if s > 4 else 0)
    status('family')

In [39]:
process_family()
combined.shape
combined.head(5)

Processing family :OK


PassengerId  Sex   Age  SibSp  Parch     Fare  Title_Master  Title_Miss  \
0            1    0  22.0      1      0   7.2500             0           0   
1            2    1  38.0      1      0  71.2833             0           0   
2            3    1  26.0      0      0   7.9250             0           1   
3            4    1  35.0      1      0  53.1000             0           0   
4            5    0  35.0      0      0   8.0500             0           0   

   Title_Mr  Title_Mrs  Title_Officer  Title_Royalty  Embarked_C  Embarked_Q  \
0         1          0              0              0           0           0   
1         0          1              0              0           1           0   
2         0          0              0              0           0           0   
3         0          1              0              0           0           0   
4         1          0              0              0           0           0   

   Embarked_S  Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  \
0           1        0        0        0        0        0        0        0   
1           0        0        0        1        0        0        0        0   
2           1        0        0        0        0        0        0        0   
3           1        0        0        1        0        0        0        0   
4           1        0        0        0        0        0        0        0   

   Cabin_T  Cabin_U  Pclass_1  Pclass_2  Pclass_3  Ticket_A  Ticket_C  \
0        0        1         0         0         1         1         0   
1        0        0         1         0         0         0         0   
2        0        1         0         0         1         0         0   
3        0        0         1         0         0         0         0   
4        0        1         0         0         1         0         0   

   Ticket_F  Ticket_L  Ticket_P  Ticket_S  Ticket_W  Ticket_XXX  FamilySize  \
0         0         0         0         0         0           0           2   
1         0         0         1         0         0           0           2   
2         0         0         0         1         0           0           1   
3         0         0         0         0         0           1           2   
4         0         0         0         0         0           1           1   

   Singleton  SmallFamily  BigFamily  
0          0            1          0  
1          0            1          0  
2          1            0          0  
3          0            1          0  
4          1            0          0

## Now it is time to Scale all of the Features

In [40]:
def scale_all_features():
    global combined
    features = list(combined.columns)
    features.remove('PassengerId')
    combined[features] = combined[features].apply(lambda x : x/x.max(), axis=0)
    print('Features scaled successfully!')

In [41]:
scale_all_features()
combined.head(5)

Features scaled successfully!


PassengerId  Sex     Age  SibSp  Parch      Fare  Title_Master  Title_Miss  \
0            1  0.0  0.2750  0.125    0.0  0.014151           0.0         0.0   
1            2  1.0  0.4750  0.125    0.0  0.139136           0.0         0.0   
2            3  1.0  0.3250  0.000    0.0  0.015469           0.0         1.0   
3            4  1.0  0.4375  0.125    0.0  0.103644           0.0         0.0   
4            5  0.0  0.4375  0.000    0.0  0.015713           0.0         0.0   

   Title_Mr  Title_Mrs  Title_Officer  Title_Royalty  Embarked_C  Embarked_Q  \
0       1.0        0.0            0.0            0.0         0.0         0.0   
1       0.0        1.0            0.0            0.0         1.0         0.0   
2       0.0        0.0            0.0            0.0         0.0         0.0   
3       0.0        1.0            0.0            0.0         0.0         0.0   
4       1.0        0.0            0.0            0.0         0.0         0.0   

   Embarked_S  Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  \
0         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1         0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0   
2         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3         1.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0   
4         1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   Cabin_T  Cabin_U  Pclass_1  Pclass_2  Pclass_3  Ticket_A  Ticket_C  \
0      0.0      1.0       0.0       0.0       1.0       1.0       0.0   
1      0.0      0.0       1.0       0.0       0.0       0.0       0.0   
2      0.0      1.0       0.0       0.0       1.0       0.0       0.0   
3      0.0      0.0       1.0       0.0       0.0       0.0       0.0   
4      0.0      1.0       0.0       0.0       1.0       0.0       0.0   

   Ticket_F  Ticket_L  Ticket_P  Ticket_S  Ticket_W  Ticket_XXX  FamilySize  \
0       0.0       0.0       0.0       0.0       0.0         0.0    0.181818   
1       0.0       0.0       1.0       0.0       0.0         0.0    0.181818   
2       0.0       0.0       0.0       1.0       0.0         0.0    0.090909   
3       0.0       0.0       0.0       0.0       0.0         1.0    0.181818   
4       0.0       0.0       0.0       0.0       0.0         1.0    0.090909   

   Singleton  SmallFamily  BigFamily  
0        0.0          1.0        0.0  
1        0.0          1.0        0.0  
2        1.0          0.0        0.0  
3        0.0          1.0        0.0  
4        1.0          0.0        0.0

In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [43]:
def compute_score(clf, X, y, scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv=5, scoring=scoring)
    return np.mean(xval)

In [55]:
def recover_train_test_target():
    global combined
    train0 = pd.read_csv('train.csv')
    targets = train0.Survived
    train = combined.loc[0:890]
    test = combined.loc[891:]
    return train, test, targets

In [56]:
train, test, targets = recover_train_test_target()

In [65]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=400)
clf = clf.fit(train, targets)

In [66]:
features = pd.DataFrame()
features['feature'] = train.columns
features['importance'] = clf.feature_importances_
features.sort_values(['importance'], ascending=False)

feature  importance
0     PassengerId    0.129971
2             Age    0.124087
8        Title_Mr    0.115402
5            Fare    0.114276
1             Sex    0.108215
7      Title_Miss    0.037721
26       Pclass_3    0.037544
9       Title_Mrs    0.037044
23        Cabin_U    0.028096
24       Pclass_1    0.024250
38      BigFamily    0.021591
35     FamilySize    0.020505
37    SmallFamily    0.019612
3           SibSp    0.018033
6    Title_Master    0.014335
4           Parch    0.013478
25       Pclass_2    0.013368
34     Ticket_XXX    0.012455
14     Embarked_S    0.012378
12     Embarked_C    0.009951
19        Cabin_E    0.009618
36      Singleton    0.008784
10  Title_Officer    0.007647
31       Ticket_P    0.007468
13     Embarked_Q    0.007293
32       Ticket_S    0.006906
16        Cabin_B    0.006869
18        Cabin_D    0.006570
17        Cabin_C    0.005826
28       Ticket_C    0.004862
27       Ticket_A    0.003801
33       Ticket_W    0.003093
15        Cabin_A    0.002624
20        Cabin_F    0.001660
21        Cabin_G    0.001510
11  Title_Royalty    0.001236
29       Ticket_F    0.001114
30       Ticket_L    0.000507
22        Cabin_T    0.000299

In [67]:
# do feature selection
model = SelectFromModel(clf, prefit=True)
train_new = model.transform(train)
train_new.shape

(891, 9)

In [68]:
test_new = model.transform(test)
test_new.shape

(418, 9)

In [69]:
forest = RandomForestClassifier(max_features='sqrt')
parameter_grid = {
    'max_depth' : [4,5,6,7,8],
    'n_estimators' : [200, 300, 400],
    'criterion' : ['gini', 'entropy']
}
# n_splits=5
cross_validation = StratifiedKFold(n_splits=5)

In [70]:
grid_search = GridSearchCV(forest, param_grid=parameter_grid, cv=cross_validation)
grid_search.fit(train_new, targets)

print('Best score : {}'.format(grid_search.best_score_))
print('Best parameters : {}'.format(grid_search.best_params_))

Best score : 0.813692480359147
Best parameters : {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 300}


In [71]:
pipeline = grid_search
output = pipeline.predict(test_new).astype(int)
df_output = pd.DataFrame()
df_output['PassengerId'] = test['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId', 'Survived']].to_csv('output.csv', index=False)